# GPT를 활용해 리뷰요약 best prompt 작성
- 우리 서비스에서 사용할 GPT모델 : gpt3.5-turbo

In [10]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# 절대경로나 정확한 상대경로 지정
load_dotenv('C:/Users/Admin/kpmg_future_lab/LLM_ex/yanolja_summary/.env', override=True)

# 환경 변수에서 API 키 가져오기
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
print(f"API Key loaded: {OPENAI_API_KEY[:10]}...")  # 앞 10글자만 출력 (확인용)

# OpenAI 클라이언트 생성
client = OpenAI(api_key=OPENAI_API_KEY)

# Chat API 호출
response = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

# 응답 출력
print(response.choices[0].message.content)


API Key loaded: sk-proj-vm...
Hello! How can I assist you today?
